In [1]:
!pip install --upgrade psycopg2

Requirement already up-to-date: psycopg2 in c:\users\caioa\anaconda3\lib\site-packages (2.9.3)


### Comandos executados no PostgreSQL

### Bibliotecas

In [2]:
import pandas as pd
import os
import psycopg2
from google.oauth2 import service_account

### Extração

In [3]:
password = os.environ.get('postgres_password')

In [4]:
conn = psycopg2.connect(
    host='localhost', 
    database='estudo',
    user='postgres', 
    password=password)

In [5]:
cur = conn.cursor()

In [6]:
query = '''
    SELECT 
        version()
'''

In [7]:
cur.execute(query)

In [8]:
cur.fetchone()

('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',)

In [9]:
query = '''
    SELECT
        *
    FROM public.users
'''

In [10]:
cur.execute(query)

In [11]:
cur.fetchall()

[(1, [{'channel': 'Facebook', 'first_name': 'Caio', 'last_name': 'Avelino'}]),
 (2, [{'channel': 'CRM', 'first_name': 'José', 'last_name': 'Ribeiro'}]),
 (3, None),
 (4, [{'channel': 'Direct', 'first_name': 'Maria', 'last_name': 'Aparecida'}]),
 (5,
  [{'channel': 'Organic', 'first_name': 'Mariane', 'last_name': 'Pereira'}]),
 (6, None),
 (7,
  [{'channel': 'Instagram', 'first_name': 'Rodrigo', 'last_name': 'Câmara'}]),
 (8, [{'channel': 'Facebook', 'first_name': 'Rafael', 'last_name': 'Santos'}]),
 (9, None),
 (10, None)]

In [21]:
df = pd.read_sql(sql=query, con=conn)

In [23]:
df

,id,user_attributes
0,1,"[{'first_name': 'Caio', 'last_name': 'Avelino'..."
1,2,"[{'first_name': 'José', 'last_name': 'Ribeiro'..."
2,3,None
3,4,"[{'first_name': 'Maria', 'last_name': 'Apareci..."
4,5,"[{'first_name': 'Mariane', 'last_name': 'Perei..."
5,6,None
6,7,"[{'first_name': 'Rodrigo', 'last_name': 'Câmar..."
7,8,"[{'first_name': 'Rafael', 'last_name': 'Santos..."
8,9,None
9,10,None


### Transformação

In [22]:
df.head()

,id,user_attributes
0,1,"[{'first_name': 'Caio', 'last_name': 'Avelino'..."
1,2,"[{'first_name': 'José', 'last_name': 'Ribeiro'..."
2,3,None
3,4,"[{'first_name': 'Maria', 'last_name': 'Apareci..."
4,5,"[{'first_name': 'Mariane', 'last_name': 'Perei..."


In [25]:
df.count()

id                 10
user_attributes     6
dtype: int64

In [24]:
df.dtypes

id                  int64
user_attributes    object
dtype: object

In [32]:
df_2 = df[df.user_attributes.isnull() == False]

In [33]:
df_2

,id,user_attributes
0,1,"[{'first_name': 'Caio', 'last_name': 'Avelino'..."
1,2,"[{'first_name': 'José', 'last_name': 'Ribeiro'..."
3,4,"[{'first_name': 'Maria', 'last_name': 'Apareci..."
4,5,"[{'first_name': 'Mariane', 'last_name': 'Perei..."
6,7,"[{'first_name': 'Rodrigo', 'last_name': 'Câmar..."
7,8,"[{'first_name': 'Rafael', 'last_name': 'Santos..."


In [35]:
list(df_2.user_attributes.apply(lambda x: x[0]))

[{'channel': 'Facebook', 'first_name': 'Caio', 'last_name': 'Avelino'},
 {'channel': 'CRM', 'first_name': 'José', 'last_name': 'Ribeiro'},
 {'channel': 'Direct', 'first_name': 'Maria', 'last_name': 'Aparecida'},
 {'channel': 'Organic', 'first_name': 'Mariane', 'last_name': 'Pereira'},
 {'channel': 'Instagram', 'first_name': 'Rodrigo', 'last_name': 'Câmara'},
 {'channel': 'Facebook', 'first_name': 'Rafael', 'last_name': 'Santos'}]

In [36]:
df_attributes = pd.DataFrame(list(df_2.user_attributes.apply(lambda x: x[0])), 
                             index=df_2.index)

In [37]:
df_attributes

,first_name,last_name,channel
0,Caio,Avelino,Facebook
1,José,Ribeiro,CRM
3,Maria,Aparecida,Direct
4,Mariane,Pereira,Organic
6,Rodrigo,Câmara,Instagram
7,Rafael,Santos,Facebook


In [51]:
df_final = df.merge(df_attributes, 
                    left_on=df.index,
                    right_on=df_attributes.index,
                    how='left')

In [52]:
df_final

,key_0,id,user_attributes,first_name,last_name,channel
0,0,1,"[{'first_name': 'Caio', 'last_name': 'Avelino'...",Caio,Avelino,Facebook
1,1,2,"[{'first_name': 'José', 'last_name': 'Ribeiro'...",José,Ribeiro,CRM
2,2,3,None,NaN,NaN,NaN
3,3,4,"[{'first_name': 'Maria', 'last_name': 'Apareci...",Maria,Aparecida,Direct
4,4,5,"[{'first_name': 'Mariane', 'last_name': 'Perei...",Mariane,Pereira,Organic
5,5,6,None,NaN,NaN,NaN
6,6,7,"[{'first_name': 'Rodrigo', 'last_name': 'Câmar...",Rodrigo,Câmara,Instagram
7,7,8,"[{'first_name': 'Rafael', 'last_name': 'Santos...",Rafael,Santos,Facebook
8,8,9,None,NaN,NaN,NaN
9,9,10,None,NaN,NaN,NaN


In [53]:
df_final.drop(columns=['key_0', 'user_attributes'], inplace=True)

In [54]:
df_final

,id,first_name,last_name,channel
0,1,Caio,Avelino,Facebook
1,2,José,Ribeiro,CRM
2,3,NaN,NaN,NaN
3,4,Maria,Aparecida,Direct
4,5,Mariane,Pereira,Organic
5,6,NaN,NaN,NaN
6,7,Rodrigo,Câmara,Instagram
7,8,Rafael,Santos,Facebook
8,9,NaN,NaN,NaN
9,10,NaN,NaN,NaN


### Carregamento

In [56]:
credentials = service_account.Credentials.from_service_account_file(filename='GBQ.json', 
                                                                    scopes=['https://www.googleapis.com/auth/cloud-platform'])

In [57]:
df_final.to_gbq(credentials=credentials, 
                destination_table='curso_etl.etl_postgres')

1it [00:07,  7.14s/it]
